# QPC nanowire spectrum

In [ ]:
from functools import partial

import adaptive
import kwant
import numpy as np
import sympy

import common
import funcs

adaptive.notebook_extension()
sympy.init_printing()

# Hamiltonian

In [ ]:
funcs.get_sympy_hamiltonian({}, 2)

# Band structure example

In [ ]:
import matplotlib.pyplot
syst = funcs.make_wire(a=10, r=50, shape='hexagon', dim=3)

In [ ]:
params = dict(alpha=0, mu=0, g=50, B_x=0, B_y=0, B_z=0, **funcs.constants)
# params['exp'] = lambda x: 1  # Uncomment this to disable the orbital effect
ham = syst.hamiltonian_submatrix(params=params)

In [ ]:
kwant.plotter.bands(syst, params=params, show=False, fig_size=(12, 6));
matplotlib.pyplot.ylim(-.1, 100);

# Calculate spectrum

In [ ]:
def minimize_wrap(x, key, lead_pars, params, band_num):
    import funcs
    from scipy.optimize import minimize_scalar
    lead = funcs.make_wire(**lead_pars)
    params[key] = x
    params = {**params, **funcs.constants}
    bands = lambda k, band_num: kwant.physics.Bands(lead, params=params)(k=k)[band_num]
    return minimize_scalar(bands, tol=1e-3, args=(band_num)).fun

In [ ]:
lead_pars = dict(a=10, r=50, shape='hexagon', dim=3)
params = dict(alpha=0, mu=0, g=50, B_x=0, B_y=0, B_z=0)
learners = []
for band_num in range(8):
    learner = adaptive.Learner1D(partial(minimize_wrap, key='B_x', lead_pars=lead_pars, params=params, band_num=band_num), (0, 2))
    learners.append(learner)
learner = adaptive.BalancingLearner(learners)
runner = adaptive.Runner(learner, goal=lambda l: l.loss() < 0.01)

In [ ]:
runner.live_info()

In [ ]:
plotter = lambda learner: learner.plot().overlay().redim(x='B_x', y='E')
runner.live_plot(plotter=plotter, update_interval=10)